In [66]:
# importing required libraries
import matplotlib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from yellowbrick.cluster import KElbowVisualizer, silhouette_visualizer
import matplotlib.pyplot as plt
from sklearn.metrics.cluster import silhouette_score, silhouette_samples, silhouette_score
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import pyvista as pv
import vtk
import os
from vtk import vtkStructuredPoints
from skimage import io, transform
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import matplotlib.colors as mcolors

---

### Professional Players

In [67]:
df = pd.read_csv("/home/bambito9/Esports-Data-Analysis/API Requests/League of Legends/Datasets/Professional/FinalProfessionalPlayersDataset.csv", dtype = {'skillSlot': str, 'buildingType': str, 'lane': str, 'monster': str, "itemName": str})
df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageDoneToChampions,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role
0,EUW1_6272362521,Blue,anonymous 2,euw1,Akshan,0.000000,554.0,581.0,1.0,0.0,...,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,TOP
1,EUW1_6272362521,Blue,anonymous 2,euw1,Akshan,1.000300,3177.0,8848.0,1.0,0.0,...,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,TOP
2,EUW1_6272362521,Blue,anonymous 2,euw1,Akshan,2.000733,2665.0,13301.0,1.0,2.0,...,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,TOP
3,EUW1_6272362521,Blue,anonymous 2,euw1,Akshan,1.932467,NaN,NaN,NaN,NaN,...,NaN,NaN,False,SkillLevelUp,1.0,NaN,NaN,NaN,NaN,TOP
4,EUW1_6272362521,Blue,anonymous 2,euw1,Akshan,3.001000,3352.0,13461.0,3.0,14.0,...,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,TOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392066,NA1_4573614609,Red,anonymous 97,na1,Kaisa,23.473867,NaN,NaN,NaN,NaN,...,NaN,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Locket of the Iron Solari,BOTTOM
392067,NA1_4573614609,Red,anonymous 97,na1,Kaisa,23.488683,NaN,NaN,NaN,NaN,...,NaN,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Control Ward,BOTTOM
392068,NA1_4573614609,Red,anonymous 97,na1,Kaisa,23.491417,NaN,NaN,NaN,NaN,...,NaN,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Control Ward,BOTTOM
392069,NA1_4573614609,Red,anonymous 97,na1,Kaisa,26.337567,NaN,NaN,NaN,NaN,...,NaN,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Null-Magic Mantle,BOTTOM


In [68]:
jungler_df = df[(df["role"] == "JUNGLE") & (df["eventType"].isna())]
jungler_df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageDoneToChampions,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role
5904,EUW1_6272528116,Red,anonymous 24,euw1,LeeSin,0.000000,14176.0,14506.0,1.0,0.0,...,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
5905,EUW1_6272528116,Red,anonymous 24,euw1,LeeSin,1.000067,8193.0,10180.0,1.0,0.0,...,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
5906,EUW1_6272528116,Red,anonymous 24,euw1,LeeSin,2.000383,10360.0,7867.0,2.0,0.0,...,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
5909,EUW1_6272528116,Red,anonymous 24,euw1,LeeSin,3.000767,11796.0,7451.0,3.0,0.0,...,0.0,18.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
5910,EUW1_6272528116,Red,anonymous 24,euw1,LeeSin,4.001050,1263.0,11706.0,3.0,0.0,...,64.0,74.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275770,KR_6376019501,Red,anonymous 49,kr,Gragas,20.005683,10601.0,7668.0,11.0,5.0,...,802.0,419.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
275772,KR_6376019501,Red,anonymous 49,kr,Gragas,21.005883,4514.0,10154.0,11.0,6.0,...,976.0,522.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
275773,KR_6376019501,Red,anonymous 49,kr,Gragas,22.006183,9666.0,3361.0,12.0,6.0,...,1028.0,522.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE
275776,KR_6376019501,Red,anonymous 49,kr,Gragas,23.006417,1082.0,1188.0,13.0,8.0,...,1353.0,1522.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE


In [69]:
champions_played = np.unique(jungler_df["champion"])
champions_played

array(['Amumu', 'AurelionSol', 'Belveth', 'Camille', 'Diana', 'Ekko',
       'Elise', 'Evelynn', 'FiddleSticks', 'Gragas', 'Graves', 'Hecarim',
       'Ivern', 'JarvanIV', 'Jayce', 'Karthus', 'Kayn', 'Khazix',
       'Kindred', 'LeeSin', 'Lillia', 'Maokai', 'MonkeyKing',
       'Mordekaiser', 'Morgana', 'Nidalee', 'Nunu', 'Poppy', 'Qiyana',
       'Rammus', 'RekSai', 'Rengar', 'Rumble', 'Ryze', 'Sejuani', 'Sylas',
       'Taliyah', 'Talon', 'Trundle', 'Vi', 'Viego', 'Volibear',
       'XinZhao', 'Zac', 'Zed'], dtype=object)

Stats on all Jungle Champions played by pros in patch 13.3

In [85]:
jungle_champions_stats = []

for champion in champions_played:
    jungle_champions_stats_row = {}
    temp_df = jungler_df[(jungler_df["champion"] == champion) & (jungler_df["timestamp"] == 0)]
    temp_df = temp_df.iloc[0, 15:35]
    jungle_champions_stats_row = temp_df
    jungle_champions_stats.append(jungle_champions_stats_row)

    

column_names = jungler_df.columns[15:35]

jungle_champions_stats_df = pd.DataFrame(jungle_champions_stats, columns=column_names)

jungle_champions_stats_df.insert(0, "champion", champions_played)
jungle_champions_stats_df

,champion,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,ccReduction,health,healthMax,...,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,power,powerMax,powerRegen,spellVamp
8764,Amumu,0.0,30.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,335.0,0.0,285.0,285.0,0.0,0.0
37544,AurelionSol,0.0,22.0,0.0,0.0,25.0,100.0,0.0,620.0,620.0,...,0.0,0.0,0.0,30.0,335.0,0.0,530.0,530.0,0.0,0.0
75835,Belveth,0.0,32.0,0.0,0.0,25.0,100.0,0.0,610.0,610.0,...,0.0,0.0,0.0,32.0,340.0,0.0,0.0,60.0,0.0,0.0
78079,Camille,0.0,35.0,0.0,0.0,25.0,100.0,0.0,646.0,646.0,...,0.0,0.0,0.0,32.0,340.0,0.0,339.0,339.0,0.0,0.0
40607,Diana,0.0,31.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,345.0,0.0,375.0,375.0,0.0,0.0
238564,Ekko,0.0,32.0,0.0,0.0,25.0,100.0,0.0,655.0,655.0,...,0.0,0.0,0.0,32.0,340.0,0.0,280.0,280.0,0.0,0.0
6151,Elise,0.0,30.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,330.0,0.0,324.0,324.0,0.0,0.0
116237,Evelynn,0.0,37.0,0.0,0.0,25.0,100.0,0.0,642.0,642.0,...,0.0,0.0,0.0,32.0,335.0,0.0,315.0,315.0,0.0,0.0
224588,FiddleSticks,0.0,34.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,335.0,0.0,500.0,500.0,0.0,0.0
9067,Gragas,0.0,38.0,0.0,0.0,25.0,100.0,0.0,670.0,670.0,...,0.0,0.0,0.0,32.0,330.0,0.0,400.0,400.0,0.0,0.0


---

### Amateur Players

In [86]:
df = pd.read_csv("/home/bambito9/Esports-Data-Analysis/API Requests/League of Legends/Datasets/Amateur/FinalAmateurPlayersDataset.csv", dtype = {'skillSlot': str, 'buildingType': str, 'lane': str, 'monster': str, "itemName": str})
df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role,rank
0,EUW1_6274275803,Blue,anonymous 125,euw1,Twitch,0.000000,329.0,650.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,UTILITY,Platinum
1,EUW1_6274275803,Blue,anonymous 125,euw1,Twitch,1.000200,6569.0,11391.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,UTILITY,Platinum
2,EUW1_6274275803,Blue,anonymous 125,euw1,Twitch,0.495733,NaN,NaN,NaN,NaN,...,NaN,False,SkillLevelUp,1.0,NaN,NaN,NaN,NaN,UTILITY,Platinum
3,EUW1_6274275803,Blue,anonymous 125,euw1,Twitch,2.000517,4320.0,12059.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,UTILITY,Platinum
4,EUW1_6274275803,Blue,anonymous 125,euw1,Twitch,1.979017,4245.0,13217.0,NaN,NaN,...,NaN,False,Assist,NaN,NaN,NaN,NaN,NaN,UTILITY,Platinum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838676,EUW1_6283483863,Red,anonymous 56,euw1,Annie,17.139450,NaN,NaN,NaN,NaN,...,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Kindlegem,UTILITY,Gold
838677,EUW1_6283483863,Red,anonymous 56,euw1,Annie,22.688683,NaN,NaN,NaN,NaN,...,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Knight's Vow,UTILITY,Gold
838678,EUW1_6283483863,Red,anonymous 56,euw1,Annie,22.826867,NaN,NaN,NaN,NaN,...,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Glacial Buckler,UTILITY,Gold
838679,EUW1_6283483863,Red,anonymous 56,euw1,Annie,25.901700,NaN,NaN,NaN,NaN,...,NaN,False,Item purchased,NaN,NaN,NaN,NaN,Kindlegem,UTILITY,Gold


#### Iron Players

In [87]:
jungler_df = df[(df["role"] == "JUNGLE") & (df["eventType"].isna()) & (df["rank"] == "Iron")]
jungler_df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role,rank
94703,EUW1_6283747670,Red,anonymous 2,euw1,Rammus,0.000000,14176.0,14506.0,1.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
94704,EUW1_6283747670,Red,anonymous 2,euw1,Rammus,1.000100,11423.0,7159.0,1.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
94706,EUW1_6283747670,Red,anonymous 2,euw1,Rammus,2.000400,12447.0,6555.0,2.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
94708,EUW1_6283747670,Red,anonymous 2,euw1,Rammus,3.000433,8043.0,9765.0,3.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
94710,EUW1_6283747670,Red,anonymous 2,euw1,Rammus,4.000467,4452.0,9634.0,4.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622409,EUW1_6289184463,Red,anonymous 15,euw1,Hecarim,24.007733,6524.0,6977.0,12.0,11.0,...,529.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
622410,EUW1_6289184463,Red,anonymous 15,euw1,Hecarim,25.007733,6011.0,6834.0,13.0,20.0,...,529.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
622412,EUW1_6289184463,Red,anonymous 15,euw1,Hecarim,26.008233,7716.0,13620.0,13.0,20.0,...,529.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron
622415,EUW1_6289184463,Red,anonymous 15,euw1,Hecarim,27.008467,11965.0,3042.0,13.0,20.0,...,529.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Iron


In [88]:
champions_played = np.unique(jungler_df["champion"])
champions_played

array(['Amumu', 'AurelionSol', 'Diana', 'Hecarim', 'JarvanIV', 'Jax',
       'Kayn', 'Kindred', 'MasterYi', 'MonkeyKing', 'Nocturne', 'Rammus',
       'Rengar', 'Shaco', 'Teemo', 'Udyr', 'Vi', 'Volibear', 'Warwick',
       'XinZhao'], dtype=object)

Stats on all Jungle Champions played by iron players in patch 13.3

In [89]:
jungle_champions_stats = []

for champion in champions_played:
    jungle_champions_stats_row = {}
    temp_df = jungler_df[(jungler_df["champion"] == champion) & (jungler_df["timestamp"] == 0)]
    temp_df = temp_df.iloc[0, 15:35]
    jungle_champions_stats_row = temp_df
    jungle_champions_stats.append(jungle_champions_stats_row)

    

column_names = jungler_df.columns[15:35]

jungle_champions_stats_df = pd.DataFrame(jungle_champions_stats, columns=column_names)

jungle_champions_stats_df.insert(0, "champion", champions_played)
jungle_champions_stats_df

,champion,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,ccReduction,health,healthMax,...,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,power,powerMax,powerRegen,spellVamp
97214,Amumu,0.0,30.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,335.0,0.0,285.0,285.0,0.0,0.0
98107,AurelionSol,0.0,22.0,0.0,0.0,25.0,100.0,0.0,620.0,620.0,...,0.0,0.0,0.0,30.0,335.0,0.0,530.0,530.0,0.0,0.0
622253,Diana,0.0,31.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,345.0,0.0,375.0,375.0,0.0,0.0
622361,Hecarim,0.0,32.0,0.0,0.0,25.0,100.0,0.0,625.0,625.0,...,0.0,0.0,0.0,32.0,345.0,0.0,277.0,277.0,0.0,0.0
98164,JarvanIV,0.0,36.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,340.0,0.0,300.0,300.0,0.0,0.0
96528,Jax,0.0,36.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,350.0,0.0,339.0,339.0,0.0,0.0
96031,Kayn,0.0,38.0,0.0,0.0,25.0,100.0,0.0,655.0,655.0,...,0.0,0.0,0.0,32.0,340.0,0.0,410.0,410.0,0.0,0.0
474575,Kindred,0.0,29.0,0.0,0.0,25.0,100.0,0.0,610.0,610.0,...,0.0,0.0,0.0,30.0,325.0,0.0,300.0,300.0,0.0,0.0
96306,MasterYi,0.0,33.0,0.0,0.0,25.0,100.0,0.0,669.0,669.0,...,0.0,0.0,0.0,32.0,355.0,0.0,251.0,251.0,0.0,0.0
571455,MonkeyKing,0.0,31.0,0.0,0.0,25.0,100.0,0.0,610.0,610.0,...,0.0,0.0,0.0,28.0,340.0,0.0,300.0,300.0,0.0,0.0


#### Bronze Players

In [90]:
jungler_df = df[(df["role"] == "JUNGLE") & (df["eventType"].isna()) & (df["rank"] == "Bronze")]
jungler_df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role,rank
372021,EUW1_6284317555,Red,anonymous 90,euw1,Amumu,0.000000,14176.0,14506.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
372022,EUW1_6284317555,Red,anonymous 90,euw1,Amumu,1.000283,11290.0,6566.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
372023,EUW1_6284317555,Red,anonymous 90,euw1,Amumu,2.000633,12330.0,6649.0,2.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
372026,EUW1_6284317555,Red,anonymous 90,euw1,Amumu,3.000767,8304.0,7554.0,3.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
372028,EUW1_6284317555,Red,anonymous 90,euw1,Amumu,4.000933,5699.0,8454.0,4.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637018,EUW1_6289551233,Blue,anonymous 106,euw1,Nunu,26.007783,7843.0,2103.0,15.0,34.0,...,1234.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
637019,EUW1_6289551233,Blue,anonymous 106,euw1,Nunu,27.008267,10737.0,10305.0,15.0,34.0,...,1234.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
637020,EUW1_6289551233,Blue,anonymous 106,euw1,Nunu,28.008400,13698.0,9821.0,17.0,37.0,...,1234.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze
637024,EUW1_6289551233,Blue,anonymous 106,euw1,Nunu,29.008467,10932.0,13166.0,17.0,43.0,...,1234.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Bronze


In [91]:
champions_played = np.unique(jungler_df["champion"])
champions_played

array(['Amumu', 'AurelionSol', 'Belveth', 'Diana', 'Ekko', 'Elise',
       'Gragas', 'Graves', 'Hecarim', 'JarvanIV', 'Jax', 'Jayce', 'Kayn',
       'Kindred', 'LeeSin', 'Lillia', 'Malphite', 'MasterYi',
       'MonkeyKing', 'Mordekaiser', 'Nidalee', 'Nunu', 'Poppy', 'Rammus',
       'RekSai', 'Rengar', 'Sejuani', 'Shaco', 'Shyvana', 'Sylas',
       'Trundle', 'Udyr', 'Vi', 'Viego', 'Volibear', 'Warwick', 'XinZhao',
       'Zac', 'Zed'], dtype=object)

Stats on all Jungle Champions played by Bronze players in patch 13.3

In [92]:
jungle_champions_stats = []

for champion in champions_played:
    jungle_champions_stats_row = {}
    temp_df = jungler_df[(jungler_df["champion"] == champion) & (jungler_df["timestamp"] == 0)]
    temp_df = temp_df.iloc[0, 15:35]
    jungle_champions_stats_row = temp_df
    jungle_champions_stats.append(jungle_champions_stats_row)

    

column_names = jungler_df.columns[15:35]

jungle_champions_stats_df = pd.DataFrame(jungle_champions_stats, columns=column_names)

jungle_champions_stats_df.insert(0, "champion", champions_played)
jungle_champions_stats_df

,champion,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,ccReduction,health,healthMax,...,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,power,powerMax,powerRegen,spellVamp
372021,Amumu,0.0,30.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,335.0,0.0,285.0,285.0,0.0,0.0
517390,AurelionSol,0.0,22.0,0.0,0.0,25.0,100.0,0.0,620.0,620.0,...,0.0,0.0,0.0,30.0,335.0,0.0,530.0,530.0,0.0,0.0
381756,Belveth,0.0,32.0,0.0,0.0,25.0,100.0,0.0,610.0,610.0,...,0.0,0.0,0.0,32.0,340.0,0.0,0.0,60.0,0.0,0.0
440493,Diana,0.0,31.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,345.0,0.0,375.0,375.0,0.0,0.0
372080,Ekko,0.0,32.0,0.0,0.0,25.0,100.0,0.0,655.0,655.0,...,0.0,0.0,0.0,32.0,340.0,0.0,280.0,280.0,0.0,0.0
386250,Elise,0.0,30.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,330.0,0.0,324.0,324.0,0.0,0.0
439760,Gragas,0.0,38.0,0.0,0.0,25.0,100.0,0.0,670.0,670.0,...,0.0,0.0,0.0,32.0,330.0,0.0,400.0,400.0,0.0,0.0
439585,Graves,0.0,33.0,0.0,0.0,25.0,100.0,0.0,625.0,625.0,...,0.0,0.0,0.0,32.0,340.0,0.0,325.0,325.0,0.0,0.0
449564,Hecarim,0.0,32.0,0.0,0.0,25.0,100.0,0.0,625.0,625.0,...,0.0,0.0,0.0,32.0,345.0,0.0,277.0,277.0,0.0,0.0
391057,JarvanIV,0.0,36.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,340.0,0.0,300.0,300.0,0.0,0.0


#### Silver Players

In [93]:
jungler_df = df[(df["role"] == "JUNGLE") & (df["eventType"].isna()) & (df["rank"] == "Silver")]
jungler_df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role,rank
17127,EUW1_6280261762,Blue,anonymous 141,euw1,Shyvana,0.000000,557.0,345.0,1.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
17128,EUW1_6280261762,Blue,anonymous 141,euw1,Shyvana,1.000533,8637.0,4630.0,1.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
17130,EUW1_6280261762,Blue,anonymous 141,euw1,Shyvana,2.000667,8697.0,2472.0,2.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
17132,EUW1_6280261762,Blue,anonymous 141,euw1,Shyvana,3.000950,3513.0,6394.0,3.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
17135,EUW1_6280261762,Blue,anonymous 141,euw1,Shyvana,4.001050,6375.0,8039.0,4.0,0.0,...,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632271,EUW1_6287909210,Red,anonymous 81,euw1,Ekko,34.009883,9606.0,5419.0,18.0,62.0,...,3685.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
632272,EUW1_6287909210,Red,anonymous 81,euw1,Ekko,35.010267,3704.0,1330.0,18.0,70.0,...,3685.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
632276,EUW1_6287909210,Red,anonymous 81,euw1,Ekko,36.010767,3441.0,2548.0,18.0,70.0,...,3822.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver
632279,EUW1_6287909210,Red,anonymous 81,euw1,Ekko,37.011133,3359.0,8454.0,18.0,72.0,...,3822.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Silver


In [94]:
champions_played = np.unique(jungler_df["champion"])
champions_played

array(['Amumu', 'Belveth', 'Diana', 'Ekko', 'Elise', 'Evelynn',
       'FiddleSticks', 'Garen', 'Gragas', 'Graves', 'Hecarim', 'Ivern',
       'JarvanIV', 'Jax', 'Kayn', 'Khazix', 'Kindred', 'LeeSin', 'Lillia',
       'Maokai', 'MasterYi', 'Mordekaiser', 'Morgana', 'Nautilus',
       'Nocturne', 'Nunu', 'Rammus', 'Rengar', 'Shyvana', 'Skarner',
       'Taliyah', 'Trundle', 'Vi', 'Viego', 'Volibear', 'Warwick',
       'XinZhao', 'Zac', 'Zeri'], dtype=object)

Stats on all Jungle Champions played by silver players in patch 13.3

In [95]:
jungle_champions_stats = []

for champion in champions_played:
    jungle_champions_stats_row = {}
    temp_df = jungler_df[(jungler_df["champion"] == champion) & (jungler_df["timestamp"] == 0)]
    temp_df = temp_df.iloc[0, 15:35]
    jungle_champions_stats_row = temp_df
    jungle_champions_stats.append(jungle_champions_stats_row)

    

column_names = jungler_df.columns[15:35]

jungle_champions_stats_df = pd.DataFrame(jungle_champions_stats, columns=column_names)

jungle_champions_stats_df.insert(0, "champion", champions_played)
jungle_champions_stats_df

,champion,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,ccReduction,health,healthMax,...,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,power,powerMax,powerRegen,spellVamp
17190,Amumu,0.0,30.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,335.0,0.0,285.0,285.0,0.0,0.0
330108,Belveth,0.0,32.0,0.0,0.0,25.0,100.0,0.0,610.0,610.0,...,0.0,0.0,0.0,32.0,340.0,0.0,0.0,60.0,0.0,0.0
286666,Diana,0.0,31.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,345.0,0.0,375.0,375.0,0.0,0.0
286839,Ekko,0.0,32.0,0.0,0.0,25.0,100.0,0.0,655.0,655.0,...,0.0,0.0,0.0,32.0,340.0,0.0,280.0,280.0,0.0,0.0
333845,Elise,0.0,30.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,330.0,0.0,324.0,324.0,0.0,0.0
286605,Evelynn,0.0,37.0,0.0,0.0,25.0,100.0,0.0,642.0,642.0,...,0.0,0.0,0.0,32.0,335.0,0.0,315.0,315.0,0.0,0.0
497144,FiddleSticks,0.0,34.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,335.0,0.0,500.0,500.0,0.0,0.0
591713,Garen,0.0,36.0,0.0,0.0,25.0,100.0,0.0,690.0,690.0,...,0.0,0.0,0.0,32.0,340.0,0.0,0.0,0.0,0.0,0.0
509523,Gragas,0.0,38.0,0.0,0.0,25.0,100.0,0.0,670.0,670.0,...,0.0,0.0,0.0,32.0,330.0,0.0,400.0,400.0,0.0,0.0
287557,Graves,0.0,33.0,0.0,0.0,25.0,100.0,0.0,625.0,625.0,...,0.0,0.0,0.0,32.0,340.0,0.0,325.0,325.0,0.0,0.0


#### Gold Players

In [96]:
jungler_df = df[(df["role"] == "JUNGLE") & (df["eventType"].isna()) & (df["rank"] == "Gold")]
jungler_df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role,rank
2866,EUW1_6274465314,Red,anonymous 129,euw1,Maokai,0.000000,14176.0,14506.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
2867,EUW1_6274465314,Red,anonymous 129,euw1,Maokai,1.000150,8948.0,9529.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
2869,EUW1_6274465314,Red,anonymous 129,euw1,Maokai,2.000667,7184.0,11119.0,2.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
2871,EUW1_6274465314,Red,anonymous 129,euw1,Maokai,3.001183,2833.0,11334.0,3.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
2873,EUW1_6274465314,Red,anonymous 129,euw1,Maokai,4.001517,11309.0,6520.0,3.0,1.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642974,EUW1_6289397856,Red,anonymous 172,euw1,Ekko,24.006683,4429.0,1177.0,14.0,30.0,...,90.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
642977,EUW1_6289397856,Red,anonymous 172,euw1,Ekko,25.007183,8729.0,3892.0,14.0,31.0,...,340.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
642980,EUW1_6289397856,Red,anonymous 172,euw1,Ekko,26.007467,11844.0,9139.0,15.0,31.0,...,340.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold
642983,EUW1_6289397856,Red,anonymous 172,euw1,Ekko,27.007617,1847.0,2066.0,15.0,35.0,...,340.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Gold


In [97]:
champions_played = np.unique(jungler_df["champion"])
champions_played

array(['Amumu', 'Belveth', 'Diana', 'Ekko', 'Elise', 'Evelynn',
       'FiddleSticks', 'JarvanIV', 'Jax', 'LeeSin', 'Maokai', 'MasterYi',
       'MonkeyKing', 'Nocturne', 'Nunu', 'Rammus', 'Singed', 'Sylas',
       'Talon', 'Udyr', 'Vi', 'Warwick', 'Zac'], dtype=object)

Stats on all Jungle Champions played by gold players in patch 13.3

In [98]:
jungle_champions_stats = []

for champion in champions_played:
    jungle_champions_stats_row = {}
    temp_df = jungler_df[(jungler_df["champion"] == champion) & (jungler_df["timestamp"] == 0)]
    temp_df = temp_df.iloc[0, 15:35]
    jungle_champions_stats_row = temp_df
    jungle_champions_stats.append(jungle_champions_stats_row)

    

column_names = jungler_df.columns[15:35]

jungle_champions_stats_df = pd.DataFrame(jungle_champions_stats, columns=column_names)

jungle_champions_stats_df.insert(0, "champion", champions_played)
jungle_champions_stats_df

,champion,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,ccReduction,health,healthMax,...,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,power,powerMax,powerRegen,spellVamp
69717,Amumu,0.0,30.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,335.0,0.0,285.0,285.0,0.0,0.0
34748,Belveth,0.0,32.0,0.0,0.0,25.0,100.0,0.0,610.0,610.0,...,0.0,0.0,0.0,32.0,340.0,0.0,0.0,60.0,0.0,0.0
567930,Diana,0.0,31.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,345.0,0.0,375.0,375.0,0.0,0.0
11025,Ekko,0.0,32.0,0.0,0.0,25.0,100.0,0.0,655.0,655.0,...,0.0,0.0,0.0,32.0,340.0,0.0,280.0,280.0,0.0,0.0
3041,Elise,0.0,30.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,330.0,0.0,324.0,324.0,0.0,0.0
69271,Evelynn,0.0,37.0,0.0,0.0,25.0,100.0,0.0,642.0,642.0,...,0.0,0.0,0.0,32.0,335.0,0.0,315.0,315.0,0.0,0.0
67680,FiddleSticks,0.0,34.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,335.0,0.0,500.0,500.0,0.0,0.0
91830,JarvanIV,0.0,36.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,340.0,0.0,300.0,300.0,0.0,0.0
67218,Jax,0.0,36.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,350.0,0.0,339.0,339.0,0.0,0.0
3368,LeeSin,0.0,36.0,0.0,0.0,25.0,100.0,0.0,645.0,645.0,...,0.0,0.0,0.0,32.0,345.0,0.0,200.0,200.0,0.0,0.0


#### Platinum Players

In [99]:
jungler_df = df[(df["role"] == "JUNGLE") & (df["eventType"].isna()) & (df["rank"] == "Platinum")]
jungler_df

,gameId,team,summoner,region,champion,timestamp,coordinate_x,coordinate_y,level,minionsKilled,...,trueDamageTaken,victory,eventType,skillSlot,buildingType,lane,monster,itemName,role,rank
45846,EUW1_6279888483,Red,anonymous 161,euw1,Vi,0.000000,14176.0,14506.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
45847,EUW1_6279888483,Red,anonymous 161,euw1,Vi,1.000017,9623.0,6322.0,1.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
45848,EUW1_6279888483,Red,anonymous 161,euw1,Vi,2.000500,6458.0,12266.0,2.0,0.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
45851,EUW1_6279888483,Red,anonymous 161,euw1,Vi,3.000900,2045.0,12826.0,3.0,1.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
45853,EUW1_6279888483,Red,anonymous 161,euw1,Vi,4.001050,12317.0,5164.0,3.0,1.0,...,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642413,EUW1_6289301059,Blue,anonymous 165,euw1,Ekko,21.005500,2476.0,8388.0,13.0,12.0,...,818.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
642415,EUW1_6289301059,Blue,anonymous 165,euw1,Ekko,22.005983,7798.0,11952.0,13.0,12.0,...,929.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
642418,EUW1_6289301059,Blue,anonymous 165,euw1,Ekko,23.005983,6993.0,9303.0,13.0,13.0,...,929.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum
642420,EUW1_6289301059,Blue,anonymous 165,euw1,Ekko,24.006150,13506.0,13848.0,14.0,13.0,...,2929.0,True,NaN,NaN,NaN,NaN,NaN,NaN,JUNGLE,Platinum


In [100]:
champions_played = np.unique(jungler_df["champion"])
champions_played

array(['Amumu', 'AurelionSol', 'Belveth', 'Diana', 'DrMundo', 'Ekko',
       'Elise', 'Evelynn', 'FiddleSticks', 'Gragas', 'Hecarim',
       'JarvanIV', 'Jax', 'Kaisa', 'Kassadin', 'Kayn', 'Khazix', 'LeeSin',
       'Lillia', 'Maokai', 'MasterYi', 'MonkeyKing', 'Mordekaiser',
       'Nautilus', 'Nidalee', 'Nocturne', 'Nunu', 'Poppy', 'Rammus',
       'RekSai', 'Rengar', 'Sejuani', 'Shaco', 'Shyvana', 'Sion', 'Sylas',
       'Taliyah', 'Trundle', 'Tryndamere', 'Udyr', 'Vi', 'Viego',
       'Warwick', 'XinZhao', 'Zac', 'Zed'], dtype=object)

Stats on all Jungle Champions played by platinum players in patch 13.3

In [101]:
jungle_champions_stats = []

for champion in champions_played:
    jungle_champions_stats_row = {}
    temp_df = jungler_df[(jungler_df["champion"] == champion) & (jungler_df["timestamp"] == 0)]
    temp_df = temp_df.iloc[0, 15:35]
    jungle_champions_stats_row = temp_df
    jungle_champions_stats.append(jungle_champions_stats_row)

    

column_names = jungler_df.columns[15:35]

jungle_champions_stats_df = pd.DataFrame(jungle_champions_stats, columns=column_names)

jungle_champions_stats_df.insert(0, "champion", champions_played)
jungle_champions_stats_df

,champion,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,ccReduction,health,healthMax,...,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,power,powerMax,powerRegen,spellVamp
53903,Amumu,0.0,30.0,0.0,0.0,25.0,100.0,0.0,685.0,685.0,...,0.0,0.0,0.0,32.0,335.0,0.0,285.0,285.0,0.0,0.0
192145,AurelionSol,0.0,22.0,0.0,0.0,25.0,100.0,0.0,620.0,620.0,...,0.0,0.0,0.0,30.0,335.0,0.0,530.0,530.0,0.0,0.0
146306,Belveth,0.0,32.0,0.0,0.0,25.0,100.0,0.0,610.0,610.0,...,0.0,0.0,0.0,32.0,340.0,0.0,0.0,60.0,0.0,0.0
54268,Diana,0.0,31.0,0.0,0.0,25.0,100.0,0.0,640.0,640.0,...,0.0,0.0,0.0,32.0,345.0,0.0,375.0,375.0,0.0,0.0
147368,DrMundo,0.0,32.0,0.0,0.0,25.0,100.0,0.0,613.0,613.0,...,0.0,0.0,0.0,29.0,345.0,0.0,0.0,0.0,0.0,0.0
55431,Ekko,0.0,32.0,0.0,0.0,25.0,100.0,0.0,655.0,655.0,...,0.0,0.0,0.0,32.0,340.0,0.0,280.0,280.0,0.0,0.0
55826,Elise,0.0,30.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,330.0,0.0,324.0,324.0,0.0,0.0
190973,Evelynn,0.0,37.0,0.0,0.0,25.0,100.0,0.0,642.0,642.0,...,0.0,0.0,0.0,32.0,335.0,0.0,315.0,315.0,0.0,0.0
54773,FiddleSticks,0.0,34.0,0.0,0.0,25.0,100.0,0.0,650.0,650.0,...,0.0,0.0,0.0,30.0,335.0,0.0,500.0,500.0,0.0,0.0
54435,Gragas,0.0,38.0,0.0,0.0,25.0,100.0,0.0,670.0,670.0,...,0.0,0.0,0.0,32.0,330.0,0.0,400.0,400.0,0.0,0.0
